# Master Analytical Table Creation – UIDAI Data Hackathon

## Overview
This notebook creates a unified master analytical table by combining the three cleaned UIDAI datasets:
- Enrollment
- Demographic Update
- Biometric Update

Each dataset represents a different category of Aadhaar service demand.
To enable comparative analysis and policy insights, the datasets are aggregated to a common analytical grain and merged into a single table.

## Analytical Grain
All datasets are aggregated at the following level:
- Date
- State

This grain balances temporal resolution with administrative stability and enables consistent cross-dataset comparison.

## Output
The final output of this notebook is:

03_Processed_Data/analysis_master_table.csv

This master table serves as the foundation for all downstream analysis and visualization.

## Step 1: Environment Setup and Library Imports

This step initializes the Python environment and imports the required libraries
for data loading, aggregation, and merging.


In [3]:
import pandas as pd
from pathlib import Path

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 120)

## Step 2: Loading Clean Processed Datasets

In this step, the cleaned and frozen datasets generated during the data preparation phase
are loaded into the analysis environment. These datasets are treated as immutable inputs
and will not be modified directly.

In [4]:
# Define processed data directory
DATA_DIR = Path("../03_Processed_Data")

# Load cleaned datasets
enrollment = pd.read_csv(DATA_DIR / "enrollment_clean.csv", parse_dates=["date"])
demographic = pd.read_csv(DATA_DIR / "demographic_update_clean.csv", parse_dates=["date"])
biometric = pd.read_csv(DATA_DIR / "biometric_update_clean.csv", parse_dates=["date"])

# Quick shape check
enrollment.shape, demographic.shape, biometric.shape

((1005293, 7), (2069863, 6), (1859634, 6))

## Step 3: Aggregating Enrollment Dataset

Enrollment records are aggregated to the (date, state) level.
Total enrollment demand is computed by summing all age-segmented enrollment counts.


In [5]:
# Aggregate enrollment data to (date, state) level
enrollment_agg = (
    enrollment
    .assign(
        total_enrollment=lambda df: (
            df["age_0_5"] + df["age_5_17"] + df["age_18_greater"]
        )
    )
    .groupby(["date", "state"], as_index=False)
    .agg(total_enrollment=("total_enrollment", "sum"))
)

# Preview result
enrollment_agg.head()

,date,state,total_enrollment
0,2025-03-02,Meghalaya,109
1,2025-03-09,Bihar,1005
2,2025-03-09,Delhi,232
3,2025-03-09,Haryana,200
4,2025-03-09,Karnataka,248


## Step 4: Aggregating Demographic Update Dataset

Demographic update records are aggregated to the (date, state) level.
Total demographic update demand is computed by summing all age-segmented
demographic update counts.


In [6]:
# Aggregate demographic update data to (date, state) level
demographic_agg = (
    demographic
    .assign(
        total_demographic_updates=lambda df: (
            df["demo_age_5_17"] + df["demo_age_17_"]
        )
    )
    .groupby(["date", "state"], as_index=False)
    .agg(total_demographic_updates=("total_demographic_updates", "sum"))
)

# Preview result
demographic_agg.head()

,date,state,total_demographic_updates
0,2025-03-01,Andaman And Nicobar Islands,1338
1,2025-03-01,Andhra Pradesh,513040
2,2025-03-01,Arunachal Pradesh,7809
3,2025-03-01,Assam,202037
4,2025-03-01,Bihar,1086699


## Step 5: Aggregating Biometric Update Dataset

Biometric update records are aggregated to the (date, state) level.
Total biometric update demand is computed by summing all age-segmented
biometric update counts.


In [7]:
# Aggregate biometric update data to (date, state) level
biometric_agg = (
    biometric
    .assign(
        total_biometric_updates=lambda df: (
            df["bio_age_5_17"] + df["bio_age_17_"]
        )
    )
    .groupby(["date", "state"], as_index=False)
    .agg(total_biometric_updates=("total_biometric_updates", "sum"))
)

# Preview result
biometric_agg.head()

,date,state,total_biometric_updates
0,2025-03-01,Andaman And Nicobar Islands,2703
1,2025-03-01,Andhra Pradesh,403296
2,2025-03-01,Arunachal Pradesh,7400
3,2025-03-01,Assam,92931
4,2025-03-01,Bihar,763509


## Step 6: Creating the Master Analytical Table

The aggregated Enrollment, Demographic Update, and Biometric Update datasets
are merged into a single master analytical table.

An outer join is performed on (date, state) to preserve all service demand signals,
including cases where certain services were active while others were not.

Missing values resulting from the join are interpreted as zero demand.


In [8]:
# Merge enrollment and demographic aggregates
master = enrollment_agg.merge(
    demographic_agg,
    on=["date", "state"],
    how="outer"
)

# Merge biometric aggregates
master = master.merge(
    biometric_agg,
    on=["date", "state"],
    how="outer"
)

# Replace missing values with zero and enforce integer type
demand_cols = [
    "total_enrollment",
    "total_demographic_updates",
    "total_biometric_updates"
]

master[demand_cols] = master[demand_cols].fillna(0).astype(int)

# Preview master table
master.head()


,date,state,total_enrollment,total_demographic_updates,total_biometric_updates
0,2025-03-01,Andaman And Nicobar Islands,0,1338,2703
1,2025-03-01,Andhra Pradesh,0,513040,403296
2,2025-03-01,Arunachal Pradesh,0,7809,7400
3,2025-03-01,Assam,0,202037,92931
4,2025-03-01,Bihar,0,1086699,763509


In [9]:
master.shape

(3851, 5)

## Step 7: Final Validation of the Master Table

This step validates the structure, data types, and numerical integrity of the master analytical table.
The objective is to confirm that the table is analysis-ready and free from structural or logical issues.


In [10]:
# Structural overview
master.info()

# Basic descriptive statistics for demand columns
master.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3851 entries, 0 to 3850
Data columns (total 5 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   date                       3851 non-null   datetime64[ns]
 1   state                      3851 non-null   object        
 2   total_enrollment           3851 non-null   int64         
 3   total_demographic_updates  3851 non-null   int64         
 4   total_biometric_updates    3851 non-null   int64         
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 150.6+ KB


,date,total_enrollment,total_demographic_updates,total_biometric_updates
count,3851,3851.000000,3.851000e+03,3.851000e+03
mean,2025-10-21 11:44:06.481433856,1410.783433,1.279428e+04,1.810024e+04
min,2025-03-01 00:00:00,0.000000,0.000000e+00,0.000000e+00
25%,2025-09-21 00:00:00,1.000000,9.950000e+01,3.100000e+01
50%,2025-11-01 00:00:00,104.000000,1.846000e+03,1.117000e+03
75%,2025-12-03 00:00:00,1290.000000,1.086650e+04,1.068050e+04
max,2025-12-31 00:00:00,157866.000000,2.199725e+06,1.426001e+06
std,NaN,4517.543170,5.612864e+04,8.114767e+04


## Step 8: Persisting the Master Analytical Table

After successful validation, the master analytical table is persisted to disk.
This table serves as the unified and authoritative dataset for all downstream
exploratory analysis, visualization, and policy interpretation.


In [12]:
# Persist master analytical table
output_path = DATA_DIR / "analysis_master_table.csv"
master.to_csv(output_path, index=False)

output_path

WindowsPath('../03_Processed_Data/analysis_master_table.csv')

In [13]:
# Quick verification of saved file
pd.read_csv(output_path, nrows=5)

,date,state,total_enrollment,total_demographic_updates,total_biometric_updates
0,2025-03-01,Andaman And Nicobar Islands,0,1338,2703
1,2025-03-01,Andhra Pradesh,0,513040,403296
2,2025-03-01,Arunachal Pradesh,0,7809,7400
3,2025-03-01,Assam,0,202037,92931
4,2025-03-01,Bihar,0,1086699,763509
